# For the speedup
### Extracting District Names into 9 Dictionaries
Because there are only nine different scene locations in the downloaded dataset.

Or if it was way more than 9, then could have sorted the file names first, then only the change of scene location, construct a new dictionary ...

In [24]:
from osgeo import ogr, osr, gdal

import fiona
from shapely.geometry import Point, shape

import numpy as np
import pandas as pd

import os
import sys
import tarfile
import timeit

In [25]:
# Change this for Win7,macOS
bases = "C:\Users\deepak\Desktop\Repo\Maps\Districts\Census\Dist.shp"
# base_ = "/Users/macbook/Documents/BTP/Satellite/Data/Maps/Districts/Census_2011"
fc = fiona.open(bases)

In [26]:
def reverse_geocode(pt):
    for feature in fc:
        if shape(feature['geometry']).contains(pt):
            return feature['properties']['DISTRICT']
    return "NRI"

In [27]:
base2 = "G:\BTP\Satellite\Data\Test"  # Win7
base = "G:\BTP\Satellite\Data\Test2"  # Win7

In [28]:
def extract(filename, force=False):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(os.path.join(base,root)) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s' % (root, filename))
    else:
        print('Extracting data for %s' % root)
        tar = tarfile.open(os.path.join(base,filename))
        sys.stdout.flush()
        tar.extractall(os.path.join(base,root))
        tar.close()        

In [29]:
# extracting for Test2
for directory, subdirList, fileList in os.walk(base):
    for filename in fileList:
        if filename.endswith(".tar.gz"): 
            d = extract(filename)

LE07_L1GT_146039_20050702_20170115_01_T2 already present - Skipping extraction of LE07_L1GT_146039_20050702_20170115_01_T2.tar.gz
LE07_L1GT_146040_20040512_20170120_01_T2 already present - Skipping extraction of LE07_L1GT_146040_20040512_20170120_01_T2.tar.gz
LE07_L1GT_146041_20041222_20170117_01_T2 already present - Skipping extraction of LE07_L1GT_146041_20041222_20170117_01_T2.tar.gz
LE07_L1GT_147039_20050725_20170114_01_T2 already present - Skipping extraction of LE07_L1GT_147039_20050725_20170114_01_T2.tar.gz
LE07_L1GT_147040_20050506_20170116_01_T2 already present - Skipping extraction of LE07_L1GT_147040_20050506_20170116_01_T2.tar.gz
LE07_L1GT_147041_20040807_20170119_01_T2 already present - Skipping extraction of LE07_L1GT_147041_20040807_20170119_01_T2.tar.gz
LE07_L1GT_148039_20050918_20170113_01_T2 already present - Skipping extraction of LE07_L1GT_148039_20050918_20170113_01_T2.tar.gz
LE07_L1GT_148040_20040627_20170120_01_T2 already present - Skipping extraction of LE07_L1G

In [30]:
directories = [os.path.join(base, d) for d in sorted(os.listdir(base)) if os.path.isdir(os.path.join(base, d))]
# print directories

In [31]:
ds = gdal.Open(base2 + "\LE07_L1TP_146039_20101223_20161211_01_T1\LE07_L1TP_146039_20101223_20161211_01_T1_B1.TIF")

In [32]:
type(ds)

osgeo.gdal.Dataset

Prepare one `ds` variable here itself, for the transformation of the coordinate system below.

In [ ]:
# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

In [33]:
def func(dsx):
    old_cs= osr.SpatialReference()
    old_cs.ImportFromWkt(dsx.GetProjectionRef())
    trs = osr.CoordinateTransformation(old_cs,new_cs) 
    return trs

In [34]:
def pixel2coord(x, y, xoff, a, b, yoff, d, e):
    """Returns global coordinates from coordinates x,y of the pixel"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [35]:
dicts = [{},{},{},{},{},{},{},{},{}]
dicti = {"146039":0, "146040":1, "146041":2, "147039":3, "147040":4, "147041":5, "148039":6, "148040":7, "149039":8}

In [36]:
k = 10

In [40]:
stx = timeit.default_timer()

for directory in directories:
    
    """ Identifying Month, Year, Spacecraft ID """
    date = directory.split('\\')[-1].split('_')[3] # Change for Win7
    satx = directory.split('\\')[-1][3]
    month = date[4:6]
    year = date[0:4]
    
    scene = directory.split('\\')[-1].split('_')[2]
    index = dicti[scene]
    
    if index != 1: continue
    
    """ Visiting every GeoTIFF file """ 
    for _,_,files in os.walk(directory):
        for filename in files:
            
            if filename.endswith(".TIF"):
                
                if filename[-5] == '2': break
                
                print os.path.join(directory,filename)
                
                ds = gdal.Open(os.path.join(directory,filename))
                if ds == None: continue
                col, row, _ = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
                xoff, a, b, yoff, d, e = ds.GetGeoTransform()
                #--------------------
                transform = func(ds)
                #--------------------
                
                """ Now go to each pixel, find its lat,lon. Hence its district, and the pixel value """
                for i in range(0,col,col/k):
                    for j in range(0,row,row/k):
                        
                        ########### fetching the lat and lon coordinates 
                        x,y = pixel2coord(i, j, xoff, a, b, yoff, d, e)
                        lonx, latx, z = transform.TransformPoint(x,y)
                        
                        
                        ########### fetching the name of district
                        
                        point = Point(lonx,latx)
                        district = reverse_geocode(point)
                        
                        dicts[index][str(lonx)+str(latx)] = district
                        
#                         #----------------------------------------------------------
#                         if filename[-5] == '1':
#                             point = Point(lonx,latx)
#                             district = reverse_geocode(point)
#                             dicts[i][str(lonx)+str(latx)] = district
#                         else:
#                             print "-------------------- !!!!!!! -------------------"
#                             district = dictx[str(lonx)+str(latx)]
#                         #----------------------------------------------------------
                        
            
                            
elapsed = timeit.default_timer() - stx
print (elapsed)
print "Seconds"

G:\BTP\Satellite\Data\Test2\LE07_L1GT_146040_20040512_20170120_01_T2\LE07_L1GT_146040_20040512_20170120_01_T2_B1.TIF


KeyboardInterrupt: 

In [41]:
print latx
print lonx
print district

28.3252362433
82.9115974967
NRI


In [42]:
print dicts[1]

{'82.925425517629.0830241588': 'NRI', '82.936151005529.6513048998': 'NRI', '82.92191839328.8935857267': 'NRI', '82.92896648529.2724568491': 'NRI', '82.67781970428.5178285689': 'NRI', '82.674834043228.328348502': 'NRI', '82.680834350328.7073030592': 'NRI', '82.671877149628.1388628844': 'NRI', '82.91500463728.5146917424': 'NRI', '82.693187268129.4651447438': 'NRI', '82.690054433329.2756928147': 'NRI', '82.696350227229.6545909699': 'NRI', '82.683878203728.8967719472': 'NRI', '82.69954354829.8440314684': 'NRI', '82.918444851528.7041415791': 'NRI', '82.932541558629.4618837714': 'NRI', '82.668948807527.9493717421': 'NRI', '82.686951488829.0862352075': 'NRI', '82.939795096429.8407202086': 'NRI'}


In [39]:
print dicts[0]

{'77.297737755429.8952824795': u'Saharanpur', '77.026737135330.4771456606': u'Ambala', '77.042552443230.084454862': u'Kurukshetra', '77.245050995731.2700202423': u'Mandi', '77.260475880830.8772761605': u'Sirmaur', '77.275601052730.4845014023': u'Sirmaur', '77.319227622929.3059964224': u'Muzaffarnagar', '77.073261020629.2989809722': u'Panipat', '76.994158574931.2624331805': u'Solan', '77.268075614830.680892595': u'Sirmaur', '77.050343895229.8880978591': u'Karnal', '77.517528759730.6878285105': u'Sirmaur', '77.018711984330.6734793671': u'Panchkula', '77.065697563729.4953608696': u'Karnal', '77.312135227129.5024324973': u'Muzaffarnagar', '77.496035241231.2771188236': u'Shimla', '77.002423380231.0661231716': u'Solan', '77.290431515330.0916963026': u'Yamunanagar', '77.304972130629.6988611971': u'Saharanpur', '77.01060762330.869805219': u'Solan', '77.034683737630.280804144': u'Ambala', '77.058058727129.6917331801': u'Karnal', '77.510434412130.8842660815': u'Shimla', '77.252801220831.07365205

In [49]:
base = "G:\BTP\Satellite\Data\Test"

In [50]:
directories = [os.path.join(base, d) for d in sorted(os.listdir(base)) if os.path.isdir(os.path.join(base, d))]
# print directories

In [51]:
ricep = pd.read_csv("C:\Users\deepak\Desktop\Repo\BTP\Ricep.csv")
ricep = ricep.drop(["Unnamed: 0"],axis=1)
ricep["value"] = ricep["Production"]/ricep["Area"]
ricep.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,phosphorus,X1,X2,X3,X4,value
0,Andhra Pradesh,anantapur,1999,kharif,Rice,37991.0,105082.0,0.0,96800.0,75400.0,643.720,881.473,2.765971
1,Andhra Pradesh,anantapur,2000,kharif,Rice,39905.0,117680.0,0.0,105082.0,96800.0,767.351,643.720,2.949004
2,Andhra Pradesh,anantapur,2001,kharif,Rice,32878.0,95609.0,0.0,117680.0,105082.0,579.338,767.351,2.907993
3,Andhra Pradesh,anantapur,2002,kharif,Rice,29066.0,66329.0,0.0,95609.0,117680.0,540.070,579.338,2.282013
4,Andhra Pradesh,anantapur,2005,kharif,Rice,25008.0,69972.0,0.0,85051.0,44891.0,819.700,564.500,2.797985


In [52]:
a = np.empty((ricep.shape[0],1))*np.NAN

In [53]:
""" 'features' contain collumn indexes for the new features """
""" 'dictn' is the dictionary mapping name of collumn index to the index number """
features = []
dictn = {}
k = 13
for i in range(1,13):
    for j in range(1,11):
        s = str(i) + "_B" + str(j) + "_"
        features.append(s+"M")
        features.append(s+"V")
        dictn[s+"M"] = k
        dictn[s+"V"] = k+1
        k = k+2

In [54]:
for i in range(1,13):
    for j in range(1,11):
        s = str(i) + "_B" + str(j) + "_"
        features.append(s+"Mn")
        features.append(s+"Vn")

In [55]:
tmp = pd.DataFrame(index=range(ricep.shape[0]),columns=features)
ricex = pd.concat([ricep,tmp], axis=1)

In [56]:
k = 50
hits = 0
times = [0,0,0,0,0,0,0,0]
nums = [0,0,0,0,0,0,0,0]

bx = False

In [57]:
stx = timeit.default_timer()

for directory in directories:
    
#     if bx: continue
#     else: bx = True
    
    """ Identifying Month, Year, Spacecraft ID """
    date = directory.split('\\')[-1].split('_')[3] # Change for Win7
    satx = directory.split('\\')[-1][3]
    month = date[4:6]
    year = date[0:4]
    
    scene = directory.split('\\')[-1].split('_')[2]
    index = dicti[scene]
    
    """ Visiting every GeoTIFF file """ 
    for _,_,files in os.walk(directory):
        for filename in files:
            
            # make sure not going into the extra folders
            
            if filename.endswith(".TIF"):
                
                if filename[-5] == '8': continue
                #--------------------------------------------------------------------------------------
                # Check for a single iteration. Which step takes the longest time.
                # improve that step
                
                # Do it all for B1.tif only.
                # for others save the row indexes found in B1's iteration
                # so dont have to search the dataframe again and again.
                # but keep track of the pixels which were not found, so have to skip them too
                # so that correct pixel value goes to the correct row in dataframe
                # have to traverse the tiff file to get the pixel values
                
                # what all steps are common for all the 10 tif files
                # the district search from the pixel's lat,lon coordinates
                # the row index search using district and the year
                # the same n is read and wrote for all the ...
                
                # If nothing works, maybe we can reduce the number of features, by just visiting 
                # First 5 TIF files for each scene.
                #--------------------------------------------------------------------------------------
                
                
                print os.path.join(directory,filename)
                
                ds = gdal.Open(os.path.join(directory,filename))
                if ds == None: continue
                col, row, _ = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
                xoff, a, b, yoff, d, e = ds.GetGeoTransform()
                
                """ Now go to each pixel, find its lat,lon. Hence its district, and the pixel value """
                """ Find the row with same (Year,District), in Crop Dataset. """
                """ Find the feature using Month, Band, SATx """
                """ For this have to find Mean & Variance """
                
                for i in range(0,col,col/k):
                    for j in range(0,row,row/k):
                        
                        st = timeit.default_timer()
                        ########### fetching the lat and lon coordinates 
                        x,y = pixel2coord(i, j, xoff, a, b, yoff, d, e)
                        lonx, latx, z = transform.TransformPoint(x,y)
                        times[0] += timeit.default_timer() - st
                        nums[0] += 1
                        
                        
                        st = timeit.default_timer()
                        ########### fetching the name of district
                        
                        district = dicts[index][str(lonx)+str(latx)]
                        #----------------------------------------------------------
                        times[1] += timeit.default_timer() - st
                        nums[1] += 1
                        
                        if district == "NRI": continue
                        
                        
                        st = timeit.default_timer()
                        ########### Locating the row in DataFrame which we want to update
                        district = district.lower()
                        district = district.strip()
                        r = ricex.index[(ricex['ind_district'] == district) & (ricex['Crop_Year'] == int(year))].tolist()
                        times[3] += timeit.default_timer() - st
                        nums[3] += 1
                        
                        
                        if len(r) == 1:
                            
                            st = timeit.default_timer()
                            ########### The pixel value for that location
                            px,py = i,j
                            pix = ds.ReadAsArray(px,py,1,1)
                            pix = pix[0][0]
                            times[2] += timeit.default_timer() - st
                            nums[2] += 1
                            
                            
                            st = timeit.default_timer()
                            """ Found the row, so now .."""
                            """ Find Collumn index corresponding to Month, Band """
                            hits = hits + 1
                            #print ("Hits: ", hits)
                            ####### Band Number ########
                            band = filename.split("\\")[-1].split("_")[7:][0].split(".")[0][1]
                            bnd = band
                            if band == '6':
                                if filename.split("\\")[-1].split("_")[7:][2][0] == '1':
                                    bnd = band
                                else:
                                    bnd = '9'
                            elif band == 'Q':
                                bnd = '10'
                            
                            sm = month + "_B" + bnd +"_M"
                            
                            cm = dictn[sm]
                            
                            r = r[0]
                            # cm is the collumn indexe for mean
                            # r[0] is the row index
                            times[4] += timeit.default_timer() - st
                            nums[4] += 1
                            
                            
                            ##### Checking if values are null ...
                            valm = ricex.iloc[r,cm]
                            if pd.isnull(valm): 
                                
                                st = timeit.default_timer()
                                ricex.iloc[r,cm] = pix
                                ricex.iloc[r,cm+1] = pix*pix
                                ricex.iloc[r,cm+240] = 1
                                times[5] += timeit.default_timer() - st
                                nums[5] += 1
                        
                                continue
                                
                                
                            st = timeit.default_timer()
                            ##### if the values are not null ...
                            valv = ricex.iloc[r,cm+1]
                            n = ricex.iloc[r,cm+240]
                            n = n+1
                            times[6] += timeit.default_timer() - st
                            nums[6] += 1
                        
                            
                            
                            st = timeit.default_timer()
                            # Mean & Variance update
                            ricex.iloc[r,cm] = valm + (pix-valm)/n
                            ricex.iloc[r,cm+1] = ((n-2)/(n-1))*valv + (pix-valm)*(pix-valm)/n
                            ricex.iloc[r,cm+240] = n
                            
                            times[7] += timeit.default_timer() - st
                            nums[7] += 1
                        
                            
                        
                            #print ("No match for the district " + district + " for the year " + year)
                            
                            
elapsed = timeit.default_timer() - stx
print (elapsed)
print "Seconds"

G:\BTP\Satellite\Data\Test\LE07_L1TP_146039_20101223_20161211_01_T1\LE07_L1TP_146039_20101223_20161211_01_T1_B1.TIF


KeyError: '76.991472364931.2515341561'

In [58]:
index

0

In [75]:
print dicts[0].keys()

['77.335882852430.1746511388', '78.579584109429.9288540221', '78.604144805230.9494805032', '78.571926699130.24262425', '79.213186585630.4099272585', '77.858564328430.8550536536', '78.88250390429.6201125941', '79.42785722929.3921850536', '78.13077293130.0767740747', '77.198819448131.1510305464', '77.105357841330.9914019989', '77.372325351730.528611124', '77.782270581630.1471491168', '77.954474060730.9750227539', '77.530926397530.2584006912', '79.281099015129.3511214835', '77.393867386329.940953595', '79.373696747229.7055476162', '79.129434514229.5845768415', '79.104752840430.8793116784', '77.659329503830.8500618352', '78.801042847631.0706938759', '77.232521335630.2893603675', '77.639235301829.9866698695', '78.160968176330.7443562214', '78.821350060330.1685283225', '77.855105671629.4035799182', '79.300096057631.1174827585', '77.316772432530.6839069342', '78.538851549429.5749392352', '78.777982653529.8931064638', '79.474650897329.5497406946', '77.079368844530.4024854286', '79.460780527630

In [76]:
print dicts[1].keys()

['84.305288166829.245492482', '84.366077193428.1834679679', '83.786079219829.3709822829', '84.878662276728.0562937492', '84.604730895729.6542898048', '84.057833637928.9104669632', '84.659615890928.5167576308', '84.167554700729.4003372619', '83.718631268828.6146487581', '83.379644402128.2420845521', '83.544550048429.2242619061', '84.106286413128.947236833', '84.415473273128.257995123', '84.594013080929.351640579', '83.782992941229.2573952087', '84.888245467328.3210771375', '83.845059972628.0437480865', '84.356468910129.357835978', '83.000337445728.0965601653', '84.690375255529.3868724686', '84.074742141629.4782692391', '84.222835087328.1111580238', '84.172532767827.9986869605', '83.597796042129.4505584828', '84.216291187229.4370445023', '84.118852849729.3636093657', '84.080487053329.6675239128', '84.873244254127.9049830287', '84.442615776729.0905357391', '84.871826787529.1545465172', '84.046828991128.5319005625', '83.968589029929.1018748428', '83.326857340527.9777178413', '83.8843796002

In [ ]:
print times
print nums
print hits

In [ ]:
for i in range(8):
    x = times[i]/nums[i]
    print (str(i) + ": " + str(x))